In [1]:
import chromadb
import asyncio
import ollama
import os
import uuid
import json

## This block is for "global-variable" setup

In [2]:
token_size = 512
shift_size = 400

## This block is for "ChromaDB" setup

In [3]:
collection = "docs"
chroma_client = chromadb.HttpClient(host='localhost', port=8000)
collection = chroma_client.get_or_create_collection(name=collection)

def heartbeat():
    return chroma_client.heartbeat()

def collection_add(i, embedding, document, metadata):
    collection.add(
        ids=[i],
        embeddings=[embedding],
        documents=[document],
        metadatas=[metadata]
    )

def collection_query(query, n_results):
    results = collection.query(
        query_embeddings=query,
        n_results=n_results
    )
    return results

def collection_reset():
    while True:
        data_ids = collection.peek()['ids']
        if not data_ids:
            break
        for i in data_ids:
            # print(i)
            collection.delete(i)

## This block is for "DB like controller"

In [5]:
def load_status(filename):
    # 檢查文件是否存在且不為空
    if not os.path.exists(filename) or os.stat(filename).st_size == 0:
        return {}
    with open(filename, 'r', encoding='utf-8') as f:
        try:
            return json.load(f)
        except json.JSONDecodeError:
            # 如果 json 文件內容格式錯誤，返回空字典
            return {}

def save_status(filename, data):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

sql_file = 'sql.json'
sql_data = load_status(sql_file)

## This block is for "RAG pipeline related" funciton
- prompt_expansion: use LLM to gen more related prompt to evaluate the response precision

In [6]:
# LLM prompt expansion >> 類似使用 "many-hit" 的方式來提高搜尋命中率
def prompt_expansion(query):
    expansive_prompts = ollama.generate(
        model="llama3",
        prompt=f"請用繁體中文，以這個 prompt: '{query}' 為基礎，以 array list 的方式，產生出額外 5 個相關的提問 prompt，我需要的 response 格式為 ['第一個產生的相似提問', '第二個產生的相似提問', '第三個產生的相似提問'] 這樣即可，不需要額外的其他內容，注意，相似提問的內容請以繁體中文呈現為主"
    )
    return expansive_prompts['response']

def query_rerank(embedding, topk):
    results = collection_query(embedding, topk)
    return results

## This block is for Application layer

In [17]:
path2dataset = './dataset/'

# To load docs from directory
def read_documents_from_directory(directory):
    documents = []
    filenames = []
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            filepath = os.path.join(directory, filename)
            with open(filepath, 'r', encoding='utf-8') as file:
                documents.append(file.read())
                filenames.append(filename)
    return documents, filenames

# To do data-embedding
def do_dataset_embedding(path2dataset, filenames, documents):
    for i, (f, d) in enumerate(zip(filenames, documents)):
        if sql_data.get(f, {}).get('status') != 'done':
            process_document(f, d)
            file_path = os.path.join(path2dataset, f)
            sql_data[f] = {'filename': f, 'path': file_path, 'status': 'done'}
        else:
            print(f"...Skipping [DONE] - {f}")

documents, filenames = read_documents_from_directory(path2dataset)

In [17]:
query = "歐洲遊學"
# expansive_prompts = prompt_expansion(query)
# print(f"extend: {expansive_prompts}")

In [18]:
embedding = ollama.embeddings(prompt=query,model="llama3")["embedding"]

In [19]:
topk = 3
results = collection_query(embedding, topk)
# print(results)
tmp_query_result = []
for d in results['documents']:
    tmp_query_result.append(d)

print(f"tmp_query_result: {tmp_query_result}")
query_justify = ollama.generate(
    model="llama3",
    prompt=f"It's a YES/No question. 如果下列於 array list [] 內的內容，有任何可以是這個問題: '{query}' 的答案，請回答 yes,否則請回答 not"
)
print(query_justify['response'])

tmp_query_result: [['意義不太大不知道發論文,所以我很早就知道幹我超討厭我超討厭盧培和所以我從來都不太會真的不太會,就很煩,都是數學然後,因為我覺得研究比較像是單點突破你要把一個細節把它做出一些新的東西可是從工程師的角度,比如說以軟體工程的角度你要去優化一個系統你有非常多選擇那這個就變成說你今天是一個,Solver Engineer是一個total solution,但你做research變成說你在某一個很特別的一個part,你必須要挖得比別人更深所以我沒辦法做應該說,其實我也不太喜歡那請問子緣,你當時是發現自己有這方面的一個天賦或興趣嗎?其實講到這個我就有點有點不好意思因為說真的,我覺得說天賦嗎?我其實沒有很有天賦,我有看過那種真的研究狂人,他是真的就是無時無刻不在做研究然後我覺得我沒有那麼我不排斥,但是我也沒有到真的很狂熱那我覺得真的會想要念博士主要還是就是有我一些想做的事情然後再加上自己其實我會喜歡看著這個東西,然後想要去把它搞懂那這個搞懂的過程,其實就像Ted說的一樣可能像工作你有很多他是一個total solution,那其實研究某方面來說同樣的問題也有很多種不同的解決方式某種程度來說這也是一種total solutio', '但是他會先讓你有一個picture讓你知道說要到那個階段大概要經歷過哪一些事情,然後你可能要做哪些準備真的是去走一條就是有些人已經幫你除過草你可以在那裡可以稍走一點彎路這也是我們創這些做這些內容分享很重要的一個原因大部分的人除非你今天是馬斯克,你要射火箭沒有人射過,不然大部分樓部都有人走過所以如果你有人可以參考那當然是很好的一件事情我們會把子緣的聯絡方式都放在下面大家有興趣可以關注他的內容那我自己覺得大家可以可能每個人都有每個人不同的一個生活的軌跡但你或許沒辦法帶走他每個部分但你可以帶走他的一個態度我覺得子緣他在很多做事的方式大家可以參考因為我們認識很久,我知道他是一個非常努力的人那這種態度是我自己認為你或許沒有人家聰明你或許沒有人家有資源但是你可以比別人做得更努力那你一樣有機會拿到你喜歡的一個成果那我知道就是在這些大家在找這些資料的時候其實你很難去找到好的管道那我剛剛說了如果有錢人走過,你有一些路可以參考那我認為是蠻重要的那最後可以請子緣幫我們介紹你的自媒體你的一些分享內容大家可以怎麼樣找到你我現在自己就是對,

## This block is testing layer
- new idea
- new way to optimize
- something new

In [ ]:
def process_document(filename, doc, model="llama3"):
    print(f"file: {filename}")
    length = len(doc)
    start = 0
    embeddings = []
    chunks = []
    i = 0

    while start < length:
        end = min(start + token_size, length)
        chunk = doc[start:end]
        # print(f"chunk: {chunk}")
        sumerize = ollama.chat(
            model = model,
            messages= [
                {
                    "role": "user",
                    "content": "我要你的回答只能是 Array-List object"
                },
                {
                    "role": "assistant",
                    "content": "[]"
                },
                {
                    "role": "user",
                    "content": f"很好，請針對下列 content:'{chunk}，產生一個 Array-List 包含屬於 content 的七個 關鍵字"
                }
            ]
        )
        print(f"sumerize: {sumerize['message']}")
        chunk_to_embedding = sumerize['message']['content']
        chunk_metadata = {"type":"podcast","name":"techporn","title":filename,"documents":chunk,"ids":i}
        # print(f"{chunk_to_embedding}")
        print(f"metadata: {chunk_metadata}")
        uid = uuid.uuid4()
        # print(f"ids:{uid}")
        print("--"*20)
        
        response = ollama.embeddings(model=model, prompt=chunk_to_embedding)
        embedding = response["embedding"]
        chunks.append(chunk_to_embedding)
        collection_add(str(uid), embedding, chunk, chunk_metadata)
        start += shift_size  # Move start forward by the shift size
        i += 1
    
    return chunks

# def do_dataset_embedding(path2dataset, filenames, documents):
#     for i, (f, d) in enumerate(zip(filenames, documents)):
#         if sql_data.get(f, {}).get('status') != 'done':
#             process_document(f, d)
#             file_path = os.path.join(path2dataset, f)
#             sql_data[f] = {'filename': f, 'path': file_path, 'status': 'done'}
#         else:
#             print(f"...Skipping [DONE] - {f}")

do_dataset_embedding(path2dataset, filenames, documents)
save_status(sql_file, sql_data)